# Доверительные интервалы для двух долей 

In [ ]:
import numpy as np
import pandas as pd

import scipy
from statsmodels.stats.weightstats import *
from statsmodels.stats.proportion import proportion_confint

## Загрузка данных

In [ ]:
data = pd.read_csv('banner_click_stat.txt', header = None, sep = '\t')
data.columns = ['banner_a', 'banner_b']

In [ ]:
data.head()

In [ ]:
data.describe()

## Интервальные оценки долей

$$\frac1{ 1 + \frac{z^2}{n} } \left( \hat{p} + \frac{z^2}{2n} \pm z \sqrt{ \frac{ \hat{p}\left(1-\hat{p}\right)}{n} + \frac{z^2}{4n^2} } \right), \;\; z \equiv z_{1-\frac{\alpha}{2}}$$ 

In [5]:
conf_interval_banner_a = proportion_confint(sum(data.banner_a), 
                                            data.shape[0],
                                            method = 'wilson')
conf_interval_banner_b = proportion_confint(sum(data.banner_b), 
                                            data.shape[0],
                                            method = 'wilson')

In [6]:
print 'interval for banner a [%f, %f]' % conf_interval_banner_a
print 'interval for banner b [%f, %f]' % conf_interval_banner_b

interval for banner a [0.026961, 0.050582]
interval for banner b [0.040747, 0.068675]


### Как их сравнить?

## Доверительный интервал для разности долей (независимые выборки)

   | $X_1$ | $X_2$  
  ------------- | -------------|
  1  | a | b 
  0  | c | d 
  $\sum$ | $n_1$| $n_2$
  
$$ \hat{p}_1 = \frac{a}{n_1}$$

$$ \hat{p}_2 = \frac{b}{n_2}$$


$$\text{Доверительный интервал для }p_1 - p_2\colon \;\; \hat{p}_1 - \hat{p}_2 \pm z_{1-\frac{\alpha}{2}}\sqrt{\frac{\hat{p}_1(1 - \hat{p}_1)}{n_1} + \frac{\hat{p}_2(1 - \hat{p}_2)}{n_2}}$$

In [33]:
def proportions_confint_diff_ind(sample1, sample2, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2.)   
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)

In [65]:
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return np.array(samples)
def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries
def shans(p1, p2):
    return np.sum(p2)/(1-np.sum(p2)) - np.sum(p1)/(1-np.sum(p1))
def ar(q,n):
    q1 = [float(1)]*q
    q0 = [float(0)]*(n-q)
    return np.append(q1,q0)
def otshans(p1,p2):
    q1 = np.sum(p1)/p1.shape[0]
    q2 = np.sum(p2)/p2.shape[0]
    print (q1/(1-q1))/(q2/(1-q2))
    return (q1/(1-q1))/(q2/(1-q2))

In [66]:
alpha = 0.05
z = scipy.stats.norm.ppf(1 - alpha / 2.)   
p2 = float(104) / float(11037)
p1 = float(189) / float(11034)

left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ 11037 + p2 * (1 - p2)/ 11034)
right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ 11037 + p2 * (1 - p2)/ 11034)

print (left_boundary, right_boundary)
print p1 - p2

(0.0046878682721335392, 0.010724179679876024)
0.007706023976


In [67]:
print p1/(1-p1)
print p2/(1-p2)
print (p1/(1-p1))/(p2/(1-p2))

0.0174273858921
0.00951248513674
1.83205394191


In [68]:
np.random.seed(0)
Xa = ar(104,11037)
Xb = ar(189,11034)

ilec_median_scores = map(otshans, get_bootstrap_samples(Xb, 1000), get_bootstrap_samples(Xa, 1000))

print ("95% confidence interval for the ILEC median repair time:",  stat_intervals(ilec_median_scores, 0.05))

2.10530717373
1.92546208978
2.21706678709
1.91264193073
2.25927419982
1.98289948956
2.06242098609
1.7681426365
2.00958949674
1.86993174425
2.37080905531
2.07289376527
2.20768984119
1.73278231563
1.83225287703
1.99687038916
1.68772422865
1.82607434091
1.77538416971
2.2147643226
1.91629472519
1.97790367226
1.94709067429
1.67787089051
1.52177196443
1.91138154257
1.99687038916
1.82813739352
2.13409503452
1.79528073208
2.0745605179
1.72913371418
2.10603499583
1.61639177099
1.9767118801
1.81854624233
1.74721534889
1.5929531437
1.95891147157
2.19285672187
1.63387603029
2.23065342912
1.8720314886
1.75321339722
1.67849571389
2.12718244151
1.7958272794
1.99521426625
2.06272196621
1.98768755923
1.82014133517
1.60753419774
1.65283158089
1.9424704975
1.84260182215
2.18171747129
2.15395944103
2.07329063684
1.59998948323
1.79028911478
1.75779726039
2.47355816026
1.70549163307
2.29115420129
1.96479729418
2.07412357514
1.85652326295
1.68780003592
2.31933101406
1.4920819657
1.75509228948
1.71766330842
1

('95% confidence interval for the ILEC median repair time:', array([ 1.46286276,  2.35093673]))


In [ ]:
print "confidence interval: [%f, %f]" % proportions_confint_diff_ind(data.banner_a, data.banner_b)

## Доверительный интервал для разности долей (связанные выборки)

  $X_1$ \ $X_2$ | 1| 0 | $\sum$
  ------------- | -------------|
  1  | e | f | e + f
  0  | g | h | g + h
  $\sum$ | e + g| f + h | n  
  
$$ \hat{p}_1 = \frac{e + f}{n}$$

$$ \hat{p}_2 = \frac{e + g}{n}$$

$$ \hat{p}_1 - \hat{p}_2 = \frac{f - g}{n}$$


$$\text{Доверительный интервал для }p_1 - p_2\colon \;\;  \frac{f - g}{n} \pm z_{1-\frac{\alpha}{2}}\sqrt{\frac{f + g}{n^2} - \frac{(f - g)^2}{n^3}}$$

In [ ]:
def proportions_confint_diff_rel(sample1, sample2, alpha = 0.05):
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    sample = zip(sample1, sample2)
    n = len(sample)
        
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    left_boundary = float(f - g) / n  - z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    right_boundary = float(f - g) / n  + z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    return (left_boundary, right_boundary)

In [ ]:
print "confidence interval: [%f, %f]" % proportions_confint_diff_rel(data.banner_a, data.banner_b)